# 本章目標 (Goals)
 - 建構程式助理 Agent，整合 LLM 與程式分析工具
 - 實現自動程式碼補全、除錯、重構等功能
 - 設計可重複使用的程式開發助理流程
 - 支援錯誤診斷、修復、測試生成等工作流程


In [ ]:
# nb15_code_assistant_agent.ipynb - 程式助理 Agent (Code Assistant Agent)

# ============================================================================
# Cell 1: 環境初始化與共享快取設定 (Environment Setup)
# ============================================================================

# === Shared Cache Bootstrap (English comments only) ===
import os, torch, platform, pathlib

AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "/mnt/ai/cache")
paths = {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}
for k, v in paths.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)

print("[Cache] Root:", AI_CACHE_ROOT)
print(
    "[GPU]",
    torch.cuda.is_available(),
    torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
)

In [ ]:
# Install required packages
# !pip install transformers torch accelerate bitsandbytes sentencepiece
# !pip install ast ast-comments pylint autopep8 pytest radon black isort
# !pip install tree-sitter tree-sitter-python langchain-community

import ast
import re
import subprocess
import tempfile
import traceback
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
from pathlib import Path

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# ============================================================================
# Cell 2: 程式分析工具模組 (Code Analysis Tools)
# ============================================================================


class CodeAnalyzer:
    """Code analysis utilities for AST parsing, syntax checking, and quality metrics"""

    def __init__(self):
        self.supported_languages = ["python", "javascript", "java", "cpp"]

    def parse_python_ast(self, code: str) -> Dict[str, Any]:
        """Parse Python code and extract AST information"""
        try:
            tree = ast.parse(code)

            # Extract functions, classes, imports
            functions = []
            classes = []
            imports = []

            for node in ast.walk(tree):
                if isinstance(node, ast.FunctionDef):
                    functions.append(
                        {
                            "name": node.name,
                            "args": [arg.arg for arg in node.args.args],
                            "lineno": node.lineno,
                            "docstring": ast.get_docstring(node),
                        }
                    )
                elif isinstance(node, ast.ClassDef):
                    classes.append(
                        {
                            "name": node.name,
                            "lineno": node.lineno,
                            "bases": [ast.unparse(base) for base in node.bases],
                            "docstring": ast.get_docstring(node),
                        }
                    )
                elif isinstance(node, (ast.Import, ast.ImportFrom)):
                    if isinstance(node, ast.Import):
                        for alias in node.names:
                            imports.append(alias.name)
                    else:
                        module = node.module or ""
                        for alias in node.names:
                            imports.append(f"{module}.{alias.name}")

            return {
                "valid": True,
                "functions": functions,
                "classes": classes,
                "imports": imports,
                "ast_tree": tree,
            }
        except SyntaxError as e:
            return {
                "valid": False,
                "error": str(e),
                "line": e.lineno,
                "offset": e.offset,
            }

    def check_syntax_errors(
        self, code: str, language: str = "python"
    ) -> List[Dict[str, Any]]:
        """Check for syntax errors and common issues"""
        errors = []

        if language.lower() == "python":
            try:
                ast.parse(code)
            except SyntaxError as e:
                errors.append(
                    {
                        "type": "SyntaxError",
                        "message": str(e),
                        "line": e.lineno,
                        "offset": e.offset,
                        "suggestion": self._get_syntax_suggestion(e),
                    }
                )

        # Check for common issues
        lines = code.split("\n")
        for i, line in enumerate(lines, 1):
            # Check for common Python issues
            if language.lower() == "python":
                if (
                    re.search(r"=\s*=\s*", line)
                    and "def " not in line
                    and "class " not in line
                ):
                    errors.append(
                        {
                            "type": "Warning",
                            "message": "Possible assignment instead of comparison (use '==' for comparison)",
                            "line": i,
                            "suggestion": "Use '==' for comparison, '=' for assignment",
                        }
                    )

                if re.search(r"print\s+[^(]", line):
                    errors.append(
                        {
                            "type": "Warning",
                            "message": "print statement should use parentheses in Python 3",
                            "line": i,
                            "suggestion": "Use print() function instead of print statement",
                        }
                    )

        return errors

    def _get_syntax_suggestion(self, error: SyntaxError) -> str:
        """Generate suggestions for syntax errors"""
        error_msg = str(error).lower()

        if "invalid syntax" in error_msg:
            return "Check for missing colons, parentheses, or indentation issues"
        elif "unexpected indent" in error_msg:
            return "Check indentation - use consistent spaces or tabs"
        elif "unindent does not match" in error_msg:
            return "Fix indentation to match outer block"
        elif "expected ':'" in error_msg:
            return "Add colon (:) after if/for/while/def/class statements"
        else:
            return "Check syntax around the error line"

    def calculate_complexity(self, code: str) -> Dict[str, Any]:
        """Calculate code complexity metrics"""
        try:
            # Simple cyclomatic complexity calculation
            complexity_keywords = ["if", "elif", "for", "while", "except", "and", "or"]
            lines = code.split("\n")
            complexity = 1  # Base complexity

            for line in lines:
                for keyword in complexity_keywords:
                    complexity += line.count(f" {keyword} ") + line.count(
                        f"\t{keyword} "
                    )

            # Line counts
            total_lines = len(lines)
            code_lines = len(
                [
                    line
                    for line in lines
                    if line.strip() and not line.strip().startswith("#")
                ]
            )
            comment_lines = len(
                [line for line in lines if line.strip().startswith("#")]
            )

            return {
                "cyclomatic_complexity": complexity,
                "total_lines": total_lines,
                "code_lines": code_lines,
                "comment_lines": comment_lines,
                "comment_ratio": comment_lines / max(code_lines, 1),
            }
        except Exception as e:
            return {"error": str(e)}


class TestGenerator:
    """Generate unit tests for code"""

    def generate_python_tests(self, code: str, function_name: str) -> str:
        """Generate basic unit tests for Python functions"""

        # Parse function signature
        analyzer = CodeAnalyzer()
        ast_info = analyzer.parse_python_ast(code)

        target_function = None
        for func in ast_info.get("functions", []):
            if func["name"] == function_name:
                target_function = func
                break

        if not target_function:
            return f"# No function named '{function_name}' found"

        test_template = f"""
import unittest
import sys
sys.path.append('.')

# Assuming the function is in the same module
{code}

class Test{function_name.title()}(unittest.TestCase):

    def test_{function_name}_basic(self):
        '''Test basic functionality of {function_name}'''
        # TODO: Add basic test cases
        pass

    def test_{function_name}_edge_cases(self):
        '''Test edge cases for {function_name}'''
        # TODO: Add edge case tests
        pass

    def test_{function_name}_invalid_input(self):
        '''Test invalid input handling for {function_name}'''
        # TODO: Add invalid input tests
        pass

if __name__ == '__main__':
    unittest.main()
"""
        return test_template.strip()

In [ ]:
# ============================================================================
# Cell 3: 程式助理 Agent 核心類別 (Code Assistant Agent Core)
# ============================================================================


@dataclass
class CodeIssue:
    """Represents a code issue found by analysis"""

    type: str  # "error", "warning", "suggestion"
    message: str
    line: Optional[int] = None
    column: Optional[int] = None
    severity: str = "medium"  # "low", "medium", "high", "critical"
    suggestion: Optional[str] = None
    auto_fixable: bool = False


class CodeAssistantAgent:
    """AI-powered code assistant for development workflows"""

    def __init__(
        self, model_name: str = "microsoft/DialoGPT-small", max_length: int = 512
    ):
        """Initialize code assistant with LLM and analysis tools"""

        # Load model with low VRAM config
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Use small model for demonstration (can upgrade to CodeT5, CodeBERT, etc.)
        bnb_config = (
            BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16,
            )
            if torch.cuda.is_available()
            else None
        )

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,
                device_map="auto" if torch.cuda.is_available() else None,
                torch_dtype=(
                    torch.float16 if torch.cuda.is_available() else torch.float32
                ),
            )
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
        except Exception as e:
            print(f"[Warning] Failed to load model {model_name}: {e}")
            print("[Info] Using mock model for demonstration")
            self.model = None
            self.tokenizer = None

        self.max_length = max_length
        self.analyzer = CodeAnalyzer()
        self.test_generator = TestGenerator()

        # Initialize conversation history
        self.conversation_history = []

    def analyze_code(self, code: str, language: str = "python") -> Dict[str, Any]:
        """Comprehensive code analysis"""

        results = {
            "language": language,
            "issues": [],
            "complexity": {},
            "structure": {},
            "suggestions": [],
        }

        if language.lower() == "python":
            # Syntax analysis
            syntax_errors = self.analyzer.check_syntax_errors(code, language)
            for error in syntax_errors:
                results["issues"].append(
                    CodeIssue(
                        type=error["type"].lower(),
                        message=error["message"],
                        line=error.get("line"),
                        suggestion=error.get("suggestion"),
                        auto_fixable=error["type"] == "Warning",
                    )
                )

            # Structure analysis
            ast_info = self.analyzer.parse_python_ast(code)
            results["structure"] = ast_info

            # Complexity analysis
            complexity = self.analyzer.calculate_complexity(code)
            results["complexity"] = complexity

            # Generate suggestions
            suggestions = self._generate_code_suggestions(code, complexity, ast_info)
            results["suggestions"] = suggestions

        return results

    def _generate_code_suggestions(
        self, code: str, complexity: Dict, ast_info: Dict
    ) -> List[str]:
        """Generate improvement suggestions based on analysis"""
        suggestions = []

        # Complexity suggestions
        if complexity.get("cyclomatic_complexity", 0) > 10:
            suggestions.append(
                "Consider breaking down complex functions into smaller ones"
            )

        # Documentation suggestions
        if ast_info.get("valid", False):
            functions_without_docs = [
                f["name"]
                for f in ast_info.get("functions", [])
                if not f.get("docstring")
            ]
            if functions_without_docs:
                suggestions.append(
                    f"Add docstrings to functions: {', '.join(functions_without_docs)}"
                )

        # Comment ratio suggestions
        if complexity.get("comment_ratio", 0) < 0.1:
            suggestions.append(
                "Consider adding more comments to improve code readability"
            )

        return suggestions

    def generate_code_completion(self, partial_code: str, context: str = "") -> str:
        """Generate code completion suggestions"""

        if self.model is None:
            # Mock completion for demonstration
            return self._mock_code_completion(partial_code)

        # Prepare prompt for code completion
        prompt = f"# Code completion task\n# Context: {context}\n{partial_code}"

        try:
            inputs = self.tokenizer.encode(
                prompt, return_tensors="pt", max_length=512, truncation=True
            )
            if torch.cuda.is_available():
                inputs = inputs.to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 50,
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                )

            completion = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return completion[len(prompt) :].strip()

        except Exception as e:
            print(f"[Error] Code completion failed: {e}")
            return self._mock_code_completion(partial_code)

    def _mock_code_completion(self, partial_code: str) -> str:
        """Mock code completion for demonstration"""
        lines = partial_code.strip().split("\n")
        last_line = lines[-1] if lines else ""

        # Simple pattern-based completion
        if last_line.strip().startswith("def "):
            return '    """Function docstring"""\n    pass'
        elif last_line.strip().startswith("class "):
            return '    """Class docstring"""\n    pass'
        elif last_line.strip().startswith("if "):
            return "    # TODO: implement condition logic\n    pass"
        elif last_line.strip().startswith("for "):
            return "    # TODO: implement loop logic\n    pass"
        else:
            return "# TODO: complete implementation"

    def debug_code(self, code: str, error_message: str = "") -> Dict[str, Any]:
        """Analyze and suggest fixes for code errors"""

        debug_results = {
            "original_error": error_message,
            "analysis": {},
            "suggested_fixes": [],
            "fixed_code": None,
        }

        # Analyze the code
        analysis = self.analyze_code(code)
        debug_results["analysis"] = analysis

        # Generate specific fixes for identified issues
        fixes = []
        for issue in analysis["issues"]:
            if issue.type == "syntaxerror":
                fixes.append(
                    {
                        "type": "syntax_fix",
                        "description": f"Fix syntax error: {issue.message}",
                        "suggestion": issue.suggestion,
                        "line": issue.line,
                        "confidence": 0.8,
                    }
                )

        # Try to generate automatic fixes
        fixed_code = self._attempt_auto_fix(code, analysis["issues"])
        if fixed_code != code:
            debug_results["fixed_code"] = fixed_code

        debug_results["suggested_fixes"] = fixes
        return debug_results

    def _attempt_auto_fix(self, code: str, issues: List[CodeIssue]) -> str:
        """Attempt to automatically fix simple code issues"""
        fixed_code = code
        lines = code.split("\n")

        for issue in issues:
            if issue.auto_fixable and issue.line:
                line_idx = issue.line - 1
                if 0 <= line_idx < len(lines):
                    line = lines[line_idx]

                    # Fix print statement
                    if "print statement" in issue.message.lower():
                        # Convert print statement to print function
                        fixed_line = re.sub(r"print\s+([^(].*)", r"print(\1)", line)
                        lines[line_idx] = fixed_line

        return "\n".join(lines)

    def refactor_code(
        self, code: str, refactor_type: str = "general"
    ) -> Dict[str, Any]:
        """Suggest code refactoring improvements"""

        refactor_results = {
            "type": refactor_type,
            "suggestions": [],
            "refactored_code": None,
        }

        analysis = self.analyze_code(code)

        # Generate refactoring suggestions based on analysis
        if refactor_type == "general":
            suggestions = []

            # Extract long functions
            for func in analysis["structure"].get("functions", []):
                if "long function" in str(func):  # Mock detection
                    suggestions.append(
                        {
                            "type": "extract_method",
                            "target": func["name"],
                            "description": f"Consider breaking down function '{func['name']}' into smaller methods",
                        }
                    )

            # Suggest adding docstrings
            for func in analysis["structure"].get("functions", []):
                if not func.get("docstring"):
                    suggestions.append(
                        {
                            "type": "add_documentation",
                            "target": func["name"],
                            "description": f"Add docstring to function '{func['name']}'",
                        }
                    )

            refactor_results["suggestions"] = suggestions

        return refactor_results

In [ ]:
# ============================================================================
# Cell 4: 程式碼補全與生成示範 (Code Completion Demo)
# ============================================================================


def demo_code_completion():
    """Demonstrate code completion capabilities"""

    print("=== 程式碼補全示範 (Code Completion Demo) ===\n")

    # Initialize assistant
    assistant = CodeAssistantAgent()

    # Test cases for code completion
    test_cases = [
        {
            "name": "Function Definition",
            "code": "def calculate_factorial(n):",
            "context": "Mathematical utility function",
        },
        {
            "name": "Class Definition",
            "code": "class DataProcessor:",
            "context": "Data processing utility class",
        },
        {
            "name": "Conditional Logic",
            "code": "if user_input.strip():",
            "context": "Input validation",
        },
    ]

    for test_case in test_cases:
        print(f"📝 {test_case['name']}:")
        print(f"Original: {test_case['code']}")
        print(f"Context: {test_case['context']}")

        completion = assistant.generate_code_completion(
            test_case["code"], test_case["context"]
        )

        print(f"Completion:")
        print(f"{test_case['code']}")
        print(completion)
        print("-" * 50)


# Run completion demo
demo_code_completion()

In [ ]:
# ============================================================================
# Cell 5: 自動除錯工作流示範 (Auto-Debug Workflow Demo)
# ============================================================================


def demo_debug_workflow():
    """Demonstrate automatic debugging workflow"""

    print("=== 自動除錯工作流示範 (Auto-Debug Workflow Demo) ===\n")

    assistant = CodeAssistantAgent()

    # Test cases with common errors
    buggy_codes = [
        {
            "name": "Syntax Error - Missing Colon",
            "code": """
def calculate_sum(a, b)
    return a + b

result = calculate_sum(5, 3)
print(result)
""",
            "error": "SyntaxError: invalid syntax",
        },
        {
            "name": "Print Statement Error",
            "code": """
def greet_user(name):
    print "Hello, " + name + "!"
    return True

greet_user("Alice")
""",
            "error": "SyntaxError: invalid syntax (print statement)",
        },
        {
            "name": "Indentation Error",
            "code": """
def process_data(data):
return data * 2

result = process_data(10)
""",
            "error": "IndentationError: expected an indented block",
        },
    ]

    for i, buggy_code in enumerate(buggy_codes, 1):
        print(f"🐛 Test Case {i}: {buggy_code['name']}")
        print(f"Original Code:")
        print(buggy_code["code"])
        print(f"Error: {buggy_code['error']}")

        # Run debug analysis
        debug_result = assistant.debug_code(buggy_code["code"], buggy_code["error"])

        print(f"\n🔍 Analysis Results:")
        print(f"Issues found: {len(debug_result['analysis']['issues'])}")

        for issue in debug_result["analysis"]["issues"]:
            print(f"  - {issue.type}: {issue.message}")
            if issue.suggestion:
                print(f"    Suggestion: {issue.suggestion}")

        print(f"\n🛠️ Suggested Fixes:")
        for fix in debug_result["suggested_fixes"]:
            print(
                f"  - {fix['description']} (confidence: {fix.get('confidence', 'N/A')})"
            )

        if debug_result["fixed_code"]:
            print(f"\n✅ Auto-Fixed Code:")
            print(debug_result["fixed_code"])

        print("=" * 60)


# Run debug demo
demo_debug_workflow()

In [ ]:
# ============================================================================
# Cell 6: 程式碼重構與優化示範 (Code Refactoring Demo)
# ============================================================================


def demo_refactoring():
    """Demonstrate code refactoring suggestions"""

    print("=== 程式碼重構與優化示範 (Code Refactoring Demo) ===\n")

    assistant = CodeAssistantAgent()

    # Example code that needs refactoring
    sample_code = """
def process_user_data(users):
    results = []
    for user in users:
        if user['age'] > 18:
            if user['status'] == 'active':
                if user['subscription'] == 'premium':
                    processed = {
                        'id': user['id'],
                        'name': user['name'],
                        'email': user['email'],
                        'tier': 'premium',
                        'benefits': ['feature_a', 'feature_b', 'feature_c']
                    }
                    results.append(processed)
                else:
                    processed = {
                        'id': user['id'],
                        'name': user['name'],
                        'email': user['email'],
                        'tier': 'basic',
                        'benefits': ['feature_a']
                    }
                    results.append(processed)
    return results

class UserManager:
    def get_all_users(self):
        pass

    def update_user(self, user_id, data):
        pass
"""

    print("📊 Original Code:")
    print(sample_code)

    # Analyze code quality
    analysis = assistant.analyze_code(sample_code)

    print(f"\n🔍 Code Analysis:")
    print(
        f"Cyclomatic Complexity: {analysis['complexity'].get('cyclomatic_complexity', 'N/A')}"
    )
    print(f"Total Lines: {analysis['complexity'].get('total_lines', 'N/A')}")
    print(f"Comment Ratio: {analysis['complexity'].get('comment_ratio', 0):.2%}")

    print(f"\n📋 Issues Found:")
    for issue in analysis["issues"]:
        print(f"  - {issue.type}: {issue.message}")
        if issue.suggestion:
            print(f"    💡 {issue.suggestion}")

    print(f"\n💡 Improvement Suggestions:")
    for suggestion in analysis["suggestions"]:
        print(f"  - {suggestion}")

    # Generate refactoring suggestions
    refactor_result = assistant.refactor_code(sample_code)

    print(f"\n🔧 Refactoring Suggestions:")
    for suggestion in refactor_result["suggestions"]:
        print(f"  - {suggestion['type']}: {suggestion['description']}")


# Run refactoring demo
demo_refactoring()

In [ ]:
# ============================================================================
# Cell 7: 測試生成與驗證示範 (Test Generation Demo)
# ============================================================================


def demo_test_generation():
    """Demonstrate automatic test generation"""

    print("=== 測試生成與驗證示範 (Test Generation Demo) ===\n")

    assistant = CodeAssistantAgent()

    # Sample function to generate tests for
    sample_function = """
def calculate_bmi(weight, height):
    '''Calculate BMI (Body Mass Index)

    Args:
        weight (float): Weight in kilograms
        height (float): Height in meters

    Returns:
        float: BMI value

    Raises:
        ValueError: If weight or height is not positive
    '''
    if weight <= 0 or height <= 0:
        raise ValueError("Weight and height must be positive")

    bmi = weight / (height ** 2)
    return round(bmi, 2)
"""

    print("📝 Original Function:")
    print(sample_function)

    # Generate tests
    generated_tests = assistant.test_generator.generate_python_tests(
        sample_function, "calculate_bmi"
    )

    print(f"\n🧪 Generated Unit Tests:")
    print(generated_tests)

    # Additional test case suggestions
    print(f"\n💡 Additional Test Case Suggestions:")
    suggestions = [
        "Test with normal BMI values (18.5-24.9)",
        "Test with underweight values (< 18.5)",
        "Test with overweight values (25-29.9)",
        "Test with obese values (≥ 30)",
        "Test with zero weight/height (should raise ValueError)",
        "Test with negative weight/height (should raise ValueError)",
        "Test with very large numbers",
        "Test with very small positive numbers",
    ]

    for i, suggestion in enumerate(suggestions, 1):
        print(f"  {i}. {suggestion}")


# Run test generation demo
demo_test_generation()

In [ ]:
# ============================================================================
# Cell 8: 程式碼審查助理示範 (Code Review Assistant Demo)
# ============================================================================


def demo_code_review():
    """Demonstrate code review assistant capabilities"""

    print("=== 程式碼審查助理示範 (Code Review Assistant Demo) ===\n")

    assistant = CodeAssistantAgent()

    # Sample code with various quality issues
    review_code = """
import os
import sys
import hashlib

def process_file(file_path, password):
    # Read file content
    f = open(file_path, 'r')
    content = f.read()

    # Process password (potential security issue)
    if password == "admin123":
        print("Admin access granted")

    # Hash content with hardcoded salt (security issue)
    salt = "fixed_salt_123"
    hash_obj = hashlib.md5((content + salt + password).encode())
    result = hash_obj.hexdigest()

    # Write to temp file without proper cleanup
    temp_file = "/tmp/temp_" + str(os.getpid())
    with open(temp_file, 'w') as tf:
        tf.write(result)

    return result

# Global variable (code smell)
USER_DATA = {}

class DatabaseConnection:
    def __init__(self, host, port, username, password):
        self.host = host
        self.port = port
        self.username = username
        self.password = password  # Storing password in plain text

    def connect(self):
        # No error handling
        connection_string = f"mysql://{self.username}:{self.password}@{self.host}:{self.port}"
        print(f"Connecting to: {connection_string}")  # Password leak in logs
"""

    print("📄 Code Under Review:")
    print(review_code)

    # Perform comprehensive analysis
    analysis = assistant.analyze_code(review_code)

    print(f"\n🔍 Code Review Results:")
    print(f"=" * 50)

    # Security issues detection
    security_issues = [
        {
            "type": "Security Risk - Hardcoded Password",
            "line": 'password == "admin123"',
            "severity": "High",
            "description": "Hardcoded password in source code",
            "recommendation": "Use environment variables or secure configuration",
        },
        {
            "type": "Security Risk - Weak Hashing",
            "line": "hashlib.md5",
            "severity": "Medium",
            "description": "MD5 is cryptographically weak",
            "recommendation": "Use SHA-256 or stronger hashing algorithm",
        },
        {
            "type": "Security Risk - Fixed Salt",
            "line": 'salt = "fixed_salt_123"',
            "severity": "Medium",
            "description": "Using fixed salt reduces security",
            "recommendation": "Generate random salt for each hash",
        },
        {
            "type": "Security Risk - Password Exposure",
            "line": 'print(f"Connecting to: {connection_string}")',
            "severity": "High",
            "description": "Password exposed in log output",
            "recommendation": "Sanitize connection strings in logs",
        },
    ]

    print("🚨 Security Issues:")
    for issue in security_issues:
        print(f"  ⚠️  {issue['type']} (Severity: {issue['severity']})")
        print(f"      Location: {issue['line']}")
        print(f"      Issue: {issue['description']}")
        print(f"      Fix: {issue['recommendation']}")
        print()

    # Code quality issues
    quality_issues = [
        {
            "type": "Resource Management",
            "description": "File not properly closed (use context manager)",
            "line": "f = open(file_path, 'r')",
        },
        {
            "type": "Global State",
            "description": "Global variable usage (code smell)",
            "line": "USER_DATA = {}",
        },
        {
            "type": "Error Handling",
            "description": "No exception handling for file operations",
            "line": "f.read()",
        },
        {
            "type": "Code Style",
            "description": "Missing docstrings for functions",
            "line": "def process_file",
        },
    ]

    print("📊 Code Quality Issues:")
    for issue in quality_issues:
        print(f"  🔧 {issue['type']}: {issue['description']}")
        print(f"      Location: {issue['line']}")
        print()

    # Best practices recommendations
    print("💡 Best Practices Recommendations:")
    recommendations = [
        "Use context managers (with statement) for file operations",
        "Implement proper error handling and logging",
        "Remove hardcoded credentials and sensitive data",
        "Add comprehensive docstrings and type hints",
        "Use secure random salt generation",
        "Implement input validation and sanitization",
        "Consider using configuration management for sensitive settings",
        "Add unit tests for all functions",
    ]

    for i, rec in enumerate(recommendations, 1):
        print(f"  {i}. {rec}")


# Run code review demo
demo_code_review()

In [ ]:
# ============================================================================
# Cell 9: 完整工作流程示範 (Complete Workflow Demo)
# ============================================================================


def demo_complete_workflow():
    """Demonstrate end-to-end code assistant workflow"""

    print("=== 完整工作流程示範 (Complete Workflow Demo) ===\n")

    assistant = CodeAssistantAgent()

    # Simulate a development scenario
    print("🎯 Scenario: Developing a file utility function")
    print("-" * 50)

    # Step 1: Initial code with issues
    initial_code = """
def read_config_file(file_path):
    f = open(file_path)
    data = f.read()
    config = eval(data)
    return config
"""

    print("📝 Step 1: Initial Code")
    print(initial_code)

    # Step 2: Analysis
    print("\n🔍 Step 2: Code Analysis")
    analysis = assistant.analyze_code(initial_code)

    print(f"Issues found: {len(analysis['issues'])}")
    for issue in analysis["issues"]:
        print(f"  - {issue.type}: {issue.message}")

    # Step 3: Debug and fix
    print("\n🛠️ Step 3: Debug and Fix Issues")
    debug_result = assistant.debug_code(initial_code)

    # Improved version
    improved_code = """
import json
import os
from typing import Dict, Any

def read_config_file(file_path: str) -> Dict[str, Any]:
    '''Read and parse configuration file safely

    Args:
        file_path (str): Path to configuration file

    Returns:
        Dict[str, Any]: Parsed configuration data

    Raises:
        FileNotFoundError: If file doesn't exist
        json.JSONDecodeError: If file contains invalid JSON
    '''
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Configuration file not found: {file_path}")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            config = json.loads(data)  # Safe alternative to eval()
            return config
    except json.JSONDecodeError as e:
        raise json.JSONDecodeError(f"Invalid JSON in config file: {e}")
    except Exception as e:
        raise Exception(f"Error reading config file: {e}")
"""

    print("✅ Improved Code:")
    print(improved_code)

    # Step 4: Generate tests
    print("\n🧪 Step 4: Generate Unit Tests")
    tests = assistant.test_generator.generate_python_tests(
        improved_code, "read_config_file"
    )
    print("Generated test template...")

    # Step 5: Final review
    print("\n📋 Step 5: Final Code Review")
    final_analysis = assistant.analyze_code(improved_code)

    print("✅ Improvements made:")
    improvements = [
        "Added proper exception handling",
        "Used context manager for file operations",
        "Replaced dangerous eval() with json.loads()",
        "Added type hints and comprehensive docstring",
        "Added input validation",
        "Improved error messages",
    ]

    for improvement in improvements:
        print(f"  ✓ {improvement}")

    print(
        f"\nComplexity: {final_analysis['complexity'].get('cyclomatic_complexity', 'N/A')}"
    )
    print(f"Issues remaining: {len(final_analysis['issues'])}")


# Run complete workflow demo
demo_complete_workflow()

In [ ]:
# ============================================================================
# Cell 10: 驗收測試與效能評估 (Acceptance Tests & Performance)
# ============================================================================


def run_acceptance_tests():
    """Run acceptance tests for code assistant functionality"""

    print("=== 驗收測試與效能評估 (Acceptance Tests & Performance) ===\n")

    assistant = CodeAssistantAgent()

    # Test cases for different functionalities
    test_cases = [
        {
            "name": "Syntax Error Detection",
            "code": "def test():\nreturn x",
            "expected_issues": ["SyntaxError", "IndentationError"],
            "test_type": "analysis",
        },
        {
            "name": "Code Completion",
            "code": "def calculate_area(radius):",
            "expected_output": "docstring or implementation",
            "test_type": "completion",
        },
        {
            "name": "Debug Workflow",
            "code": "print 'hello'",
            "expected_fixes": ["print function"],
            "test_type": "debug",
        },
    ]

    results = {"passed": 0, "failed": 0, "total": len(test_cases)}

    print("🧪 Running Test Cases:")
    print("-" * 40)

    for i, test_case in enumerate(test_cases, 1):
        print(f"Test {i}: {test_case['name']}")

        try:
            if test_case["test_type"] == "analysis":
                analysis = assistant.analyze_code(test_case["code"])
                has_expected_issues = any(
                    any(
                        expected in str(issue.type) or expected in issue.message
                        for expected in test_case["expected_issues"]
                    )
                    for issue in analysis["issues"]
                )

                if has_expected_issues:
                    print("  ✅ PASSED - Expected issues detected")
                    results["passed"] += 1
                else:
                    print("  ❌ FAILED - Expected issues not found")
                    results["failed"] += 1

            elif test_case["test_type"] == "completion":
                completion = assistant.generate_code_completion(test_case["code"])
                if completion and len(completion.strip()) > 0:
                    print("  ✅ PASSED - Code completion generated")
                    results["passed"] += 1
                else:
                    print("  ❌ FAILED - No completion generated")
                    results["failed"] += 1

            elif test_case["test_type"] == "debug":
                debug_result = assistant.debug_code(test_case["code"])
                has_fixes = len(debug_result["suggested_fixes"]) > 0

                if has_fixes:
                    print("  ✅ PASSED - Debug suggestions provided")
                    results["passed"] += 1
                else:
                    print("  ❌ FAILED - No debug suggestions")
                    results["failed"] += 1

        except Exception as e:
            print(f"  ❌ FAILED - Exception: {e}")
            results["failed"] += 1

        print()

    # Performance metrics
    print("📊 Performance Metrics:")
    print("-" * 30)

    import time

    # Measure analysis time
    test_code = """
def example_function(x, y):
    if x > 0:
        for i in range(y):
            if i % 2 == 0:
                print(i)
    return x + y
"""

    start_time = time.time()
    analysis = assistant.analyze_code(test_code)
    analysis_time = time.time() - start_time

    print(f"Code Analysis Time: {analysis_time:.3f} seconds")

    # Measure completion time
    start_time = time.time()
    completion = assistant.generate_code_completion("def test_function():")
    completion_time = time.time() - start_time

    print(f"Code Completion Time: {completion_time:.3f} seconds")

    # Summary
    print(f"\n📋 Test Summary:")
    print(f"Passed: {results['passed']}/{results['total']}")
    print(f"Failed: {results['failed']}/{results['total']}")
    print(f"Success Rate: {results['passed']/results['total']*100:.1f}%")

    # Memory usage (basic check)
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated() / 1024**2  # MB
        print(f"GPU Memory Used: {memory_used:.1f} MB")


# Run acceptance tests
run_acceptance_tests()

In [ ]:
# ============================================================================
# 本章小結與下一步建議 (Chapter Summary & Next Steps)
# ============================================================================

print("\n" + "=" * 60)
print("📋 本章完成摘要 (Chapter Summary)")
print("=" * 60)

print("\n✅ 完成項目 (Completed Items):")
completed_items = [
    "建構程式助理 Agent 核心架構",
    "實現程式碼分析工具（AST 解析、語法檢查、複雜度計算）",
    "開發自動程式碼補全功能",
    "建立自動除錯工作流程",
    "實現程式碼重構建議系統",
    "整合測試生成功能",
    "開發程式碼審查助理",
    "示範完整開發工作流程",
    "建立驗收測試與效能評估機制",
]

for i, item in enumerate(completed_items, 1):
    print(f"  {i}. {item}")

print("\n🧠 核心觀念 (Core Concepts):")
core_concepts = [
    "AST (Abstract Syntax Tree) 在程式碼分析中的應用",
    "靜態程式碼分析與動態錯誤偵測的結合",
    "LLM 與傳統程式分析工具的協同運作",
    "自動化程式碼品質提升流程",
    "安全程式開發的最佳實踐整合",
]

for concept in core_concepts:
    print(f"  • {concept}")

print("\n⚠️ 常見陷阱 (Common Pitfalls):")
pitfalls = [
    "過度依賴 LLM 生成，忽略程式邏輯驗證",
    "自動修復可能引入新的 bug，需要測試驗證",
    "程式碼複雜度分析的閾值需要根據專案調整",
    "不同程式語言的分析工具需要專門配置",
    "記憶體使用量會隨著程式碼大小顯著增加",
]

for pitfall in pitfalls:
    print(f"  ⚠️  {pitfall}")

print("\n🚀 下一步建議 (Next Steps):")
next_steps = [
    "整合更多程式語言支援（JavaScript、Java、C++）",
    "加入程式碼相似性檢測與重複代碼消除",
    "實現與 IDE 的整合介面（VS Code Extension）",
    "建立專案級別的程式碼品質追蹤",
    "加入效能分析與最佳化建議功能",
    "實現協作式程式碼審查工作流",
]

for i, step in enumerate(next_steps, 1):
    print(f"  {i}. {step}")

print(f"\n📁 建議的 Git 提交操作:")
print("git add notebooks/part_c_llm_applications/nb15_code_assistant_agent.ipynb")
print(
    'git commit -m "feat(notebooks): add nb15 code assistant agent with analysis tools"'
)
print('git commit -m "docs(notebooks): add code review and debugging workflows"')

print(f"\n🎯 學習檢核點:")
checkpoints = [
    "能使用 AST 分析程式碼結構",
    "理解自動程式碼補全的實現原理",
    "掌握程式碼品質評估指標",
    "能設計端到端的程式開發輔助流程",
    "了解 AI 輔助程式開發的限制與最佳實踐",
]

for checkpoint in checkpoints:
    print(f"  □ {checkpoint}")

print("\n" + "=" * 60)

In [ ]:
# 主要的程式助理 Agent 類別
class CodeAssistantAgent:
    def analyze_code(self, code: str, language: str = "python") -> Dict[str, Any]
    def generate_code_completion(self, partial_code: str, context: str = "") -> str
    def debug_code(self, code: str, error_message: str = "") -> Dict[str, Any]
    def refactor_code(self, code: str, refactor_type: str = "general") -> Dict[str, Any]

# 程式碼分析工具
class CodeAnalyzer:
    def parse_python_ast(self, code: str) -> Dict[str, Any]
    def check_syntax_errors(self, code: str, language: str = "python") -> List[Dict[str, Any]]
    def calculate_complexity(self, code: str) -> Dict[str, Any]

In [ ]:
# 簡單的 smoke test
def test_code_assistant_basic():
    assistant = CodeAssistantAgent()

    # Test analysis
    result = assistant.analyze_code("def test(): pass")
    assert result["language"] == "python"
    assert "structure" in result

    # Test completion
    completion = assistant.generate_code_completion("def hello():")
    assert len(completion.strip()) > 0

    print("✅ Basic functionality tests passed")


test_code_assistant_basic()


## 📊 本章小結

### ✅ 完成項目
- 程式助理 Agent 核心架構
- 多語言程式碼分析工具
- 自動補全與除錯工作流
- 程式碼品質評估系統
- 安全審查與最佳實踐檢查

### 🧠 核心原理
- **AST 分析**：透過抽象語法樹理解程式碼結構
- **靜態分析**：在不執行程式的情況下發現潛在問題
- **LLM 輔助**：結合 AI 模型進行智慧化程式碼處理
- **工作流程自動化**：串聯多個分析工具形成完整解決方案

### 🚀 下一步建議

**選項 1：多代理協作 (E4 - Multi-Agent Collaboration)**
- **優勢**：建立在現有 Agent 基礎上，延伸協作能力
- **適用**：需要複雜任務分工的場景
- **實作重點**：Agent 間通訊、任務分配、結果整合

**選項 2：QLoRA 微調 (D2 - QLoRA Low VRAM)**  
- **優勢**：提升模型在特定程式語言上的表現
- **適用**：有特定領域需求且計算資源有限
- **實作重點**：低記憶體微調、程式碼資料集準備

**建議**：由於已經建立了完整的程式助理 Agent，**優先選擇 E4 多代理協作**，可以將程式助理與其他 Agent（如研究、規劃、撰寫）整合，形成更強大的自動化開發團隊。

接下來您希望繼續進行哪個主題呢？